# Infer-13-Debugging : Troubleshooting et Bonnes Pratiques

**Serie** : Programmation Probabiliste avec Infer.NET (Bonus)  
**Duree estimee** : 45 minutes  
**Prerequis** : Avoir explore plusieurs notebooks de la serie

---

## Objectifs

- Diagnostiquer les problemes courants d'inference
- Comparer les algorithmes (EP, VMP, Gibbs)
- Utiliser les outils de debug d'Infer.NET
- Appliquer les bonnes pratiques de modelisation

---

## Navigation

| Retour | Index |
|--------|-------|
| [Infer-12-Recommenders](Infer-12-Recommenders.ipynb) | [README](README.md) |

---

## 1. Configuration

In [ ]:
#r "nuget: Microsoft.ML.Probabilistic"
#r "nuget: Microsoft.ML.Probabilistic.Compiler"

using Microsoft.ML.Probabilistic;
using Microsoft.ML.Probabilistic.Distributions;
using Microsoft.ML.Probabilistic.Utilities;
using Microsoft.ML.Probabilistic.Math;
using Microsoft.ML.Probabilistic.Models;
using Microsoft.ML.Probabilistic.Algorithms;
using Microsoft.ML.Probabilistic.Compiler;

Console.WriteLine("Infer.NET pret !");

## 2. Erreurs Courantes et Solutions

### 2.1 Catalogue des Erreurs

| Erreur | Cause | Solution |
|--------|-------|----------|
| `Model has no support` | Observation impossible sous le prior | Elargir le prior ou verifier les observations |
| `Improper distribution` | Divergence de l'inference | Utiliser des priors plus informatifs |
| `Could not find method` | Operation non supportee | Reformuler avec des operations de base |
| `Compilation timeout` | Modele trop complexe | Simplifier ou compiler separement |
| `Memory exceeded` | Trop de variables | Reduire la taille ou utiliser des arrays |

In [ ]:
// Exemple 1 : Erreur "Model has no support"

Console.WriteLine("=== Erreur : Model has no support ===");
Console.WriteLine();

// PROBLEME : Observer une valeur impossible sous le prior
try
{
    Variable<double> x = Variable.GaussianFromMeanAndPrecision(0, 1000);  // Prior tres concentre autour de 0
    x.ObservedValue = 100;  // Observation tres eloignee
    
    InferenceEngine engine = new InferenceEngine();
    engine.Compiler.CompilerChoice = CompilerChoice.Roslyn;
    var result = engine.Infer(x);  // Peut echouer ou donner des resultats etranges
}
catch (Exception e)
{
    Console.WriteLine($"Erreur : {e.Message.Substring(0, Math.Min(100, e.Message.Length))}...");
}

// SOLUTION : Utiliser un prior plus large
Console.WriteLine("\nSOLUTION : Elargir le prior");
Variable<double> x2 = Variable.GaussianFromMeanAndPrecision(50, 0.01);  // Prior large
x2.ObservedValue = 100;

InferenceEngine engine2 = new InferenceEngine();
engine2.Compiler.CompilerChoice = CompilerChoice.Roslyn;
Console.WriteLine($"Resultat avec prior large : {engine2.Infer(x2)}");

In [ ]:
// Exemple 2 : Probleme de convergence (posterieurs uniformes)

Console.WriteLine("=== Probleme : Convergence vers mode symetrique ===");
Console.WriteLine();

// PROBLEME : Priors symetriques dans un modele de melange
int nComp = 2;
Range compRange = new Range(nComp);

VariableArray<double> means = Variable.Array<double>(compRange);
means[compRange] = Variable.GaussianFromMeanAndPrecision(0, 0.01).ForEach(compRange);  // Prior identique

Console.WriteLine("Avec priors symetriques (meme pour toutes les composantes) :");
Console.WriteLine("  -> Le modele peut converger vers une solution ou toutes");
Console.WriteLine("     les composantes sont identiques (mode symetrique).");

// SOLUTION : Priors asymetriques
Console.WriteLine("\nSOLUTION : Utiliser des priors asymetriques");
Console.WriteLine("  means[0] ~ Gaussian(-5, 1)");
Console.WriteLine("  means[1] ~ Gaussian(+5, 1)");
Console.WriteLine("  -> Force les composantes a etre distinctes");

## 3. Comparaison des Algorithmes d'Inference

### Quand utiliser quel algorithme ?

| Algorithme | Forces | Faiblesses | Usage recommande |
|------------|--------|------------|------------------|
| **EP** | Rapide, bon pour melanges continus | Peut diverger, approximatif | Modeles Gaussian, Probit |
| **VMP** | Stable, bon pour discret | Sous-estime l'incertitude | LDA, melanges categoriques |
| **Gibbs** | Exact asymptotiquement | Lent, diagnostics difficiles | Validation, petits modeles |

In [ ]:
// Comparaison EP vs VMP sur un modele simple

Console.WriteLine("=== Comparaison EP vs VMP ===");
Console.WriteLine();

// Modele : estimation de moyenne avec observations bruitees
double[] observations = { 2.1, 1.9, 2.3, 2.0, 1.8, 2.2 };
int nObs = observations.Length;

// Fonction pour creer et inferer le modele
Gaussian InferAvecAlgo(IAlgorithm algo, string nom)
{
    Variable<double> mean = Variable.GaussianFromMeanAndPrecision(0, 0.01);
    Variable<double> prec = Variable.GammaFromShapeAndScale(2, 0.5);
    
    Range r = new Range(nObs);
    VariableArray<double> obs = Variable.Array<double>(r);
    obs[r] = Variable.GaussianFromMeanAndPrecision(mean, prec).ForEach(r);
    obs.ObservedValue = observations;
    
    InferenceEngine engine = new InferenceEngine(algo);
    engine.Compiler.CompilerChoice = CompilerChoice.Roslyn;
    engine.ShowProgress = false;
    
    return engine.Infer<Gaussian>(mean);
}

var resultEP = InferAvecAlgo(new ExpectationPropagation(), "EP");
var resultVMP = InferAvecAlgo(new VariationalMessagePassing(), "VMP");

Console.WriteLine($"Observations : {string.Join(", ", observations)}");
Console.WriteLine($"Moyenne empirique : {observations.Average():F3}");
Console.WriteLine();
Console.WriteLine($"EP  : {resultEP}");
Console.WriteLine($"VMP : {resultVMP}");
Console.WriteLine();
Console.WriteLine("Note : VMP tend a avoir une variance plus faible (sous-estime l'incertitude)");

## 4. Outils de Debug Infer.NET

### 4.1 Options du Moteur

```csharp
engine.ShowProgress = true;           // Affiche les iterations
engine.ShowSchedule = true;           // Affiche l'ordre des messages
engine.ShowFactorGraph = true;        // Genere le graphe de facteurs
engine.Compiler.WriteSourceFiles = true;  // Sauvegarde le code genere
engine.Compiler.ShowWarnings = true;  // Affiche les avertissements
```

In [ ]:
// Demonstration des outils de debug

Console.WriteLine("=== Outils de Debug ===");
Console.WriteLine();

// Modele simple pour demonstration
Variable<double> mu = Variable.GaussianFromMeanAndPrecision(0, 1).Named("mu");
Variable<double> y = Variable.GaussianFromMeanAndPrecision(mu, 1).Named("y");
y.ObservedValue = 5.0;

InferenceEngine debugEngine = new InferenceEngine();
debugEngine.Compiler.CompilerChoice = CompilerChoice.Roslyn;

// Activer les options de debug
debugEngine.ShowProgress = true;
debugEngine.Compiler.ShowWarnings = true;

Console.WriteLine("Options de debug activees :");
Console.WriteLine("  - ShowProgress : affiche les iterations");
Console.WriteLine("  - ShowWarnings : affiche les avertissements du compilateur");
Console.WriteLine();

var muPost = debugEngine.Infer<Gaussian>(mu);
Console.WriteLine($"\nResultat : mu ~ {muPost}");

## 5. Bonnes Pratiques de Modelisation

### 5.1 Nommage des Variables

```csharp
// BON : Noms explicites avec .Named()
Variable<double> capaciteEtudiant = Variable.GaussianFromMeanAndPrecision(0, 1).Named("capacite");

// MAUVAIS : Variables anonymes
Variable<double> x = Variable.GaussianFromMeanAndPrecision(0, 1);
```

### 5.2 Priors Informatifs

| Situation | Prior recommande |
|-----------|------------------|
| Moyenne inconnue | Gaussian large (precision ~0.01) |
| Precision inconnue | Gamma(2, 0.5) ou plus concentre |
| Probabilite | Beta(1, 1) pour uniforme, Beta(2, 2) pour centre |
| Poids melange | Dirichlet(1, 1, ...) pour uniforme |

In [ ]:
// Demonstration de l'importance des priors

Console.WriteLine("=== Impact du choix des Priors ===");
Console.WriteLine();

// Observations : 3 succes sur 5 essais
int succes = 3, echecs = 2;

// Differents priors pour la probabilite
var priors = new (string nom, double a, double b)[] {
    ("Uniforme Beta(1,1)", 1, 1),
    ("Centre Beta(2,2)", 2, 2),
    ("Informatif Beta(5,5)", 5, 5),
    ("Biaise succes Beta(8,2)", 8, 2)
};

Console.WriteLine($"Observations : {succes} succes, {echecs} echecs");
Console.WriteLine($"MLE (maximum de vraisemblance) : {(double)succes/(succes+echecs):F2}");
Console.WriteLine();

foreach (var (nom, a, b) in priors)
{
    Variable<double> p = Variable.Beta(a, b);
    Variable<int> obs = Variable.Binomial(succes + echecs, p);
    obs.ObservedValue = succes;
    
    InferenceEngine eng = new InferenceEngine();
    eng.Compiler.CompilerChoice = CompilerChoice.Roslyn;
    eng.ShowProgress = false;
    
    Beta postP = eng.Infer<Beta>(p);
    Console.WriteLine($"{nom,-25} -> Posterieur : mean = {postP.GetMean():F3}");
}

Console.WriteLine();
Console.WriteLine("Observation : Le prior influence le posterieur, surtout avec peu de donnees.");

## 6. Checklist de Debugging

Quand votre modele ne fonctionne pas, verifiez :

### Etape 1 : Verification du Modele
- [ ] Les types des variables sont corrects (double vs int vs bool)
- [ ] Les observations sont dans le support du prior
- [ ] Les arrays ont les bonnes dimensions
- [ ] Les Range sont correctement definis

### Etape 2 : Verification de l'Inference
- [ ] L'algorithme est adapte au modele (EP/VMP/Gibbs)
- [ ] Le nombre d'iterations est suffisant
- [ ] Les warnings de compilation sont examines

### Etape 3 : Verification des Resultats
- [ ] Les posterieurs ne sont pas degeneres (variance > 0)
- [ ] Les moyennes sont dans des plages raisonnables
- [ ] Les predictions sur donnees connues sont correctes

In [ ]:
// Fonction utilitaire de diagnostic

void DiagnosticPosterior<T>(T posterior, string nom) where T : IDistribution<double>
{
    Console.WriteLine($"=== Diagnostic : {nom} ===");
    Console.WriteLine($"  Distribution : {posterior}");
    Console.WriteLine($"  Moyenne : {posterior.GetMean():F4}");
    
    double variance = 0;
    if (posterior is Gaussian g) variance = g.GetVariance();
    else if (posterior is Gamma ga) variance = ga.GetVariance();
    else if (posterior is Beta be) variance = be.GetVariance();
    
    Console.WriteLine($"  Variance : {variance:F6}");
    
    // Avertissements
    if (variance < 1e-10)
        Console.WriteLine("  [ALERTE] Variance tres faible - possible degenerescence");
    if (variance > 1e6)
        Console.WriteLine("  [ALERTE] Variance tres elevee - inference non informative");
    Console.WriteLine();
}

// Exemple d'utilisation
Variable<double> test = Variable.GaussianFromMeanAndPrecision(0, 0.1);
Variable<double> yTest = Variable.GaussianFromMeanAndPrecision(test, 1);
yTest.ObservedValue = 3.0;

InferenceEngine diagEngine = new InferenceEngine();
diagEngine.Compiler.CompilerChoice = CompilerChoice.Roslyn;
diagEngine.ShowProgress = false;

var testPost = diagEngine.Infer<Gaussian>(test);
DiagnosticPosterior(testPost, "test");

## 7. Exercice : Debugger un Modele

### Enonce

Le modele ci-dessous a plusieurs problemes. Identifiez et corrigez-les.

In [ ]:
// EXERCICE : Trouvez les problemes dans ce modele

Console.WriteLine("=== Exercice : Debugger ce modele ===");
Console.WriteLine();

// Probleme 1 : Prior trop etroit pour les observations
// Probleme 2 : Variables non nommees
// Probleme 3 : Precision negative (invalide)

// Modele original (avec erreurs)
/*
Variable<double> m = Variable.GaussianFromMeanAndPrecision(0, 100);  // Prior trop concentre
Variable<double> p = Variable.GammaFromShapeAndScale(0.1, 0.1);      // Shape trop petit
Variable<double> obs = Variable.GaussianFromMeanAndPrecision(m, p);
obs.ObservedValue = 50;  // Tres eloigne du prior sur m
*/

// Version corrigee
Variable<double> moyenne = Variable.GaussianFromMeanAndPrecision(25, 0.01).Named("moyenne");  // Prior large
Variable<double> precision = Variable.GammaFromShapeAndScale(2, 0.5).Named("precision");      // Shape >= 1
Variable<double> observation = Variable.GaussianFromMeanAndPrecision(moyenne, precision).Named("obs");
observation.ObservedValue = 50;

InferenceEngine exEngine = new InferenceEngine();
exEngine.Compiler.CompilerChoice = CompilerChoice.Roslyn;
exEngine.ShowProgress = false;

var moyPost = exEngine.Infer<Gaussian>(moyenne);
var precPost = exEngine.Infer<Gamma>(precision);

Console.WriteLine("Corrections appliquees :");
Console.WriteLine("1. Prior sur moyenne : precision 0.01 (large) au lieu de 100 (etroit)");
Console.WriteLine("2. Prior sur precision : Gamma(2, 0.5) au lieu de Gamma(0.1, 0.1)");
Console.WriteLine("3. Variables nommees avec .Named()");
Console.WriteLine();
Console.WriteLine($"Resultats : moyenne ~ {moyPost}, precision ~ {precPost}");

## 8. Resume

| Probleme | Symptome | Solution |
|----------|----------|----------|
| **Prior trop etroit** | "No support" ou posterieurs etranges | Elargir le prior |
| **Mode symetrique** | Posterieurs uniformes | Priors asymetriques |
| **Divergence** | Valeurs infinies ou NaN | Changer d'algorithme ou regulariser |
| **Lenteur** | Compilation longue | Simplifier le modele, cacher les types |

---

## Ressources

- [Documentation Infer.NET](https://dotnet.github.io/infer/)
- [FAQ Troubleshooting](https://dotnet.github.io/infer/userguide/Frequently%20Asked%20Questions.html)
- [GitHub Issues](https://github.com/dotnet/infer/issues)